<a href="https://colab.research.google.com/github/petrum01/object-detection/blob/master/Train_with_DarkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install dependencies**: numpy, python-opencv, cython etc.

In [1]:
!apt-get update
!pip3 install numpy
!apt-get install python-opencv -y
!pip install cython

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [83.2 kB]
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [903 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [6,933 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [625 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 

**Install DarkFlow**

In [2]:
# Clean up the directory
%cd /content/
%rm -rf darkflow
# Clone the darkflow repository
!git clone https://github.com/thtrieu/darkflow
# Change into the darkflow dir and install darkflow with pip
%cd darkflow
!python setup.py build_ext --inplace

/content
Cloning into 'darkflow'...
remote: Enumerating objects: 2706, done.
remote: Total 2706 (delta 0), reused 0 (delta 0), pack-reused 2706
Receiving objects: 100% (2706/2706), 18.74 MiB | 17.17 MiB/s, done.
Resolving deltas: 100% (1781/1781), done.
/content/darkflow
Compiling darkflow/cython_utils/nms.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo2_findboxes.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo_findboxes.pyx because it changed.
[1/3] Cythonizing darkflow/cython_utils/cy_yolo2_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/darkflow/darkflow/cython_utils/cy_yolo2_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/3] Cythonizing darkflow/cython_utils/cy_yolo_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cyth

Download pretrained weights. tiny-yolo-voc.weights

In [3]:
%cd /content/darkflow/
%rm -rf weights
%mkdir weights
%cd weights/
!wget https://oc.codespring.ro/s/Jgyo6N4Jen3ma2P/download
%mv download tiny-yolo-voc.weights

/content/darkflow
/content/darkflow/weights
--2019-01-10 08:10:10--  https://oc.codespring.ro/s/Jgyo6N4Jen3ma2P/download
Resolving oc.codespring.ro (oc.codespring.ro)... 188.213.217.24
Connecting to oc.codespring.ro (oc.codespring.ro)|188.213.217.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63471556 (61M) [application/octet-stream]
Saving to: ‘download’

download            100%[===================>]  60.53M  4.58MB/s    in 22s     

2019-01-10 08:10:32 (2.81 MB/s) - ‘download’ saved [63471556/63471556]



Download my modified configuration file.
* in the last layer : the number of classes should be changed from 80 to 1: classes=80 => classes=1
* in the convolutional layer above (the second to last layer) to num * (classes + 5) (30 in this case)

In [4]:
# use Pydrive to download your file from google drive
!pip install PyDrive

    100% |████████████████████████████████| 993kB 23.4MB/s 
  Running setup.py bdist_wheel for PyDrive ... - done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
%cd /content/darkflow/cfg

# download custom cfg file
file_id = '1hCAtZTaacqM6VPmmTvb-EYErvsv7JaSA' # URL id. 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('tiny-yolo-voc-1c.cfg')

/content/darkflow/cfg


**Copy labels to labels.txt**

In [7]:
%cd /content/darkflow/

# download labels.txt file
file_id = '1QMF1slBVO8KzkAJW0IEvXLvbtuCqPedp' # URL id. 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('labels.txt')

/content/darkflow


**Download data set**

In [8]:
%cd /content/darkflow/

file_id = '1uNurF5g1WWHiSVAmx55t-9w_Uwr-ruPL' # URL id. 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('input.tar')
# untar to directory data
!tar xf input.tar

/content/darkflow


In [11]:
# download PAscal VOC annotations
%cd /content/darkflow/

file_id = '1Qyvhgh-0jWRuCcgA7WUxgJVyuSaPvgoZ' # URL id. 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('PASCAL_VOC.tar')
# untar
!tar xf PASCAL_VOC.tar

/content/darkflow


In [49]:
# download last checkpoints
%cd /content/darkflow/
file_id = '1TDmotm2vK6kAHEtxpgT4l7NSIG4-0mxh' # URL id. 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('ckpt.tar')
# untar
!tar xf ckpt.tar

/content/darkflow


In [72]:
%cd /content/darkflow/ckpt
file_id = '1v7cWD-6AYqxpj6JG-pS5XyvOsNBdwdhq' # URL id. 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('checkpoint')
!ls -a


/content/darkflow/ckpt
.
..
cfg
checkpoint
tiny-yolo-voc-1c-6.data-00000-of-00001
tiny-yolo-voc-1c-6.index
tiny-yolo-voc-1c-6.meta
tiny-yolo-voc-1c-6.profile
tiny-yolo-voc-1c-7812.data-00000-of-00001
tiny-yolo-voc-1c-7812.index
tiny-yolo-voc-1c-7812.meta
tiny-yolo-voc-1c-7812.profile


**Start training**

In [74]:
%%time
#start training from the last checkpoint
%cd /content/darkflow
!./flow --model cfg/tiny-yolo-voc-1c.cfg --train --dataset "/content/darkflow/input" --annotation "/content/darkflow/PASCAL_VOC" --gpu 1 --epoch 1000 --save 100 --load -1

/content/darkflow

Parsing cfg/tiny-yolo-voc-1c.cfg
Loading None ...
Finished in 8.702278137207031e-05s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 416, 416, 3)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 16)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 16)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 32)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 52, 52, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 26, 26, 128)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 26, 26, 256)
 Load  |  Yep!  | maxp 2x2p0_2           

In [76]:
# Download the checkpoints
%cd /content/darkflow/ckpt
!ls -a
from google.colab import files
files.download('checkpoint')
files.download('tiny-yolo-voc-1c-10962.data-00000-of-00001')
files.download('tiny-yolo-voc-1c-10962.index')
files.download('tiny-yolo-voc-1c-10962.meta')
files.download('tiny-yolo-voc-1c-10962.profile')


/content/darkflow/ckpt
.					    tiny-yolo-voc-1c-8208.profile
..					    tiny-yolo-voc-1c-8214.profile
cfg					    tiny-yolo-voc-1c-8220.profile
checkpoint				    tiny-yolo-voc-1c-8226.profile
tiny-yolo-voc-1c-10002.profile		    tiny-yolo-voc-1c-8232.profile
tiny-yolo-voc-1c-10008.profile		    tiny-yolo-voc-1c-8238.profile
tiny-yolo-voc-1c-10014.profile		    tiny-yolo-voc-1c-8244.profile
tiny-yolo-voc-1c-10020.profile		    tiny-yolo-voc-1c-8250.profile
tiny-yolo-voc-1c-10026.profile		    tiny-yolo-voc-1c-8256.profile
tiny-yolo-voc-1c-10032.profile		    tiny-yolo-voc-1c-8262.profile
tiny-yolo-voc-1c-10038.profile		    tiny-yolo-voc-1c-8268.profile
tiny-yolo-voc-1c-10044.profile		    tiny-yolo-voc-1c-8274.profile
tiny-yolo-voc-1c-10050.profile		    tiny-yolo-voc-1c-8280.profile
tiny-yolo-voc-1c-10056.profile		    tiny-yolo-voc-1c-8286.profile
tiny-yolo-voc-1c-10062.profile		    tiny-yolo-voc-1c-8292.profile
tiny-yolo-voc-1c-10068.profile		    tiny-yolo-voc-1c-8298.profile
tiny-yolo-voc-

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 33842, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

FileNotFoundError: ignored

In [0]:
%cd /content/darkflow
!./flow --model cfg/tiny-yolo-voc-1c.cfg --load -1 --savepb

In [67]:
# Download the protobuf file
%cd /content/darkflow/built_graph
!ls
from google.colab import files
files.download('tiny-yolo-voc-1c.meta')
files.download('tiny-yolo-voc-1c.pb')

/content/darkflow/built_graph
tiny-yolo-voc-1c.meta  tiny-yolo-voc-1c.pb
